In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd "/content/drive/MyDrive/Capstone_Project"
!ls

/content/drive/MyDrive/Capstone_Project
annotations
checkpoints
COCO_train2014_000000581909.jpg
error
image_captioning_3_model_train_test_mscoco2014_10k_data.ipynb
image_captioning_3_model_train_test_mscoco2014_1k_data.ipynb
image_captioning_3_model_train_test_mscoco2014_20k_data_100_epoch.ipynb
image_captioning_3_model_train_test_mscoco2014_20k_data_150_epoch.ipynb
image_captioning_3_model_train_test_mscoco2014_20k_data_300_epoch.ipynb
image_captioning_3_model_train_test_mscoco2014_20k_data_50_epoch.ipynb
image_captioning_3_model_train_test_mscoco2014_20k_data.ipynb
image_captioning_3_model_train_test_mscoco2014_50k_data.ipynb
image_captioning_load_model_text_similarity_v1.ipynb
image_captioning_load_model_text_similarity_v2.ipynb
materials
old
output_10k
output_1k
output_20k
output_20k_100_epoch
output_20k_150_epoch
output_20k_20_epoch
output_20k_300_epoch
output_20k_50_epoch
output_50k
output_nlp
sentence_similarity_custom_model_A_v1.ipynb
sentence_similarity_custom_model_B_v2.ipynb

In [3]:
!pip install -q sentence_transformers
!pip install -q huggingface-hub
!pip install datasets

     |████████████████████████████████| 79 kB 3.5 MB/s 
     |████████████████████████████████| 4.0 MB 9.1 MB/s 
     |████████████████████████████████| 1.2 MB 46.5 MB/s 
     |████████████████████████████████| 77 kB 6.8 MB/s 
     |████████████████████████████████| 6.5 MB 27.4 MB/s 
     |████████████████████████████████| 895 kB 81.9 MB/s 
     |████████████████████████████████| 596 kB 43.6 MB/s 
     |████████████████████████████████| 325 kB 4.1 MB/s 
     |████████████████████████████████| 1.1 MB 60.1 MB/s 
     |████████████████████████████████| 212 kB 64.9 MB/s 
     |████████████████████████████████| 136 kB 72.5 MB/s 
     |████████████████████████████████| 127 kB 64.6 MB/s 
     |████████████████████████████████| 271 kB 57.1 MB/s 
     |████████████████████████████████| 94 kB 3.1 MB/s 
     |████████████████████████████████| 144 kB 36.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninst

In [4]:
from datasets import load_dataset

snli = load_dataset("snli", split="train")
dataset = snli
snli

Downloading:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.9M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

Dataset snli downloaded and prepared to /root/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b. Subsequent calls will reuse this data.


Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 550152
})

In [5]:
print(snli[0])
print(snli[1])

{'premise': 'A person on a horse jumps over a broken down airplane.', 'hypothesis': 'A person is training his horse for a competition.', 'label': 1}
{'premise': 'A person on a horse jumps over a broken down airplane.', 'hypothesis': 'A person is at a diner, ordering an omelette.', 'label': 2}


In [6]:
print(f"Number of entries in dataset BEFORE removal of invalid entries: {len(dataset)}")
dataset = dataset.filter(
    lambda x: False if x['label'] == -1 else True
)
print(f"Number of entries in dataset AFTER removal of invalid entries: {len(dataset)}")

Number of entries in dataset BEFORE removal of invalid entries: 550152


  0%|          | 0/551 [00:00<?, ?ba/s]

Number of entries in dataset AFTER removal of invalid entries: 549367


In [7]:
from sentence_transformers import InputExample
from tqdm.auto import tqdm

train_samples = []
for row in tqdm(dataset):
  train_samples.append(InputExample(
      texts=[row['premise'], row['hypothesis']],
      label=row['label']
  ))

  0%|          | 0/549367 [00:00<?, ?it/s]

In [8]:
from torch.utils.data import DataLoader

batch_size = 16
loader = DataLoader(train_samples, batch_size=batch_size, shuffle=True)

In [9]:
# https://www.youtube.com/watch?v=aSx0jg9ZILo

from sentence_transformers import SentenceTransformer, models

bert = models.Transformer('bert-base-uncased')
pooler = models.Pooling(
    bert.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True
)

model = SentenceTransformer(modules=[bert, pooler])
model

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [10]:
from sentence_transformers import losses

loss = losses.SoftmaxLoss(
    model=model,
    sentence_embedding_dimension=model.get_sentence_embedding_dimension(),
    num_labels=3
)

In [11]:
epochs = 1
warmup_steps = int(0.1 * len(dataset))
TRAINING_MODE = False

if TRAINING_MODE:
  model.fit(
      train_objectives=[(loader, loss)],
      epochs=epochs,
      warmup_steps=warmup_steps,
      output_path="./output_nlp/sbert_test_b",
      show_progress_bar=True
  )

In [12]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from pprint import pprint

In [13]:
model = SentenceTransformer('./output_nlp/sbert_test_b')
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [14]:
output = "That is a happy person"
input_text = "That is a happy dog"
text2 = "That is a very happy person"
text3 = "I see an animal"

sentences = [output, input_text, text2, text3]

sentence_embeddings = model.encode(sentences)

In [15]:
# for sentence, embedding in zip(sentences, sentence_embeddings):
#     print("Sentence:", sentence)
#     print("Embedding:", embedding)
#     print("")

In [16]:
len(sentence_embeddings[0])

768

In [18]:
for i in range(1, len(sentences)):
  pprint('Similarity between {} and {} is {}'.format(sentences[0],
       sentences[i],
       cosine_similarity(sentence_embeddings[0].reshape(1, -1),
       sentence_embeddings[i].reshape(1, -1))[0][0]))

('Similarity between That is a happy person and That is a happy dog is '
 '0.8367971181869507')
('Similarity between That is a happy person and That is a very happy person is '
 '0.9715514183044434')
('Similarity between That is a happy person and I see an animal is '
 '0.5928057432174683')
